In [2]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
year = '2024'
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text=f"absolute Verteilung der Beeinsträchtigungen",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [2]:
df = pd.read_excel('data.xlsx', sheet_name=year)
df = df[(df['Variable'] == 'Studienbeeinträchtigungen') | (df['Category'] == 'Anzahl Teilnahmen an Studierendenbefragung') | (df['Category'] == 'Anteil Teilnehmer (%)')]
df = df[df.columns[~df.columns.str.contains(r'\*')]] # drop * in the data
df.columns = df.columns.str.replace('*', '', regex=False)# just to be sure
df = df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo

clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]


In [3]:
df = df[df.columns[~df.columns.str.contains(r'\*')]] # drop * in the data
df.columns = df.columns.str.replace('*', '', regex=False)# just to be sure
df = df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo
clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

# smack together the similar col names 
#merged_df = df[['Jahr']].copy()
#for clean_name, original_cols in clean_columns.items():
#    if len(original_cols) > 1:
#        merged_df[clean_name] = df[original_cols].sum(axis=1)
#    else:
#        merged_df[clean_name] = df[original_cols[0]]
#df = merged_df.fillna(value=0) # use if needed for plotting

In [4]:
df['sum'] = df.sum(axis=1, numeric_only=True)
df = df.loc[~df['Category'].isin(['Anzahl Teilnahmen an Studierendenbefragung', 'Anteil Teilnehmer (%)']), ['Category', 'sum']]
df['Category'] = df['Category'].str.replace(' (%)', '', regex=False)


In [5]:
umbenennungen = {
    'zeitliche Überschneidungen': 'Zeitmanagement-Probleme',
    'Stundenplanprobleme wegen weiter Wege': 'Wegeprobleme',
    'Organisation des Prüfungsablaufes': 'Prüfungsorganisation',
    'Engpässe in der Literaturversorgung': 'Literaturmangel',
    'kein Labor- / Praktikumsplatz bekommen': 'Fehlender Praktikum-/Laborsplatz',
    'Anforderungen sind zu hoch': 'Überforderung',
    'Leistungsnachweise / Prüfungen nicht geschafft': 'Prüfungsprobleme',
    'Job / Erwerbstätigkeit': 'Erwerbstätigkeit',
    'familiäre Verpflichtungen': 'Familienpflichten',
    'Krankheit / Behinderung': 'Gesundheitsprobleme',
    'große Entfernung zwischen Wohnort und JLU': 'Entfernung zum Studienort',
    'Doppelstudium': 'Doppelte Studienbelastung',
    'Nachholen von erforderlichen Vorkenntnissen': 'Vorkenntnisnachholung',
    'fehlende soziale Kontakte': 'Isolation',
    'Benachteiligung': 'Ungleichbehandlung',
    'fehlende Berufsperspektiven': 'Karriereunsicherheit'
}

# Erstelle ein DataFrame
df_ = pd.DataFrame(df, columns=['Category'])

# Ersetze die Werte in der Spalte 'Category' basierend auf dem Dictionary
df['Category'] = df['Category'].replace(umbenennungen)


In [6]:
fig = px.line_polar(df, r=df['sum'], theta=df['Category'], line_close=True)
fig.update_traces(fill='toself')
fig.update_layout(
    template='infoviz',
    title=f'Beeinträchtigende Faktoren im Studiums des FB09 in {year}',
    polar=dict(
        radialaxis=dict(  
            range=[0, 800]  
        )
    )
)


fig.show()

In [3]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text=f"normierte, relative Verteilung der Beeinsträchtigungen",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [56]:
import pandas as pd

# Daten einlesen
df = pd.read_excel('data.xlsx', sheet_name=year)

# Nur relevante Zeilen für Studienbeeinträchtigungen, Anzahl Teilnahmen an Studierendenbefragung und Anteil Teilnehmer (%)
df = df[(df['Variable'] == 'Studienbeeinträchtigungen') | 
        (df['Category'] == 'Anzahl Teilnahmen an Studierendenbefragung') | 
        (df['Category'] == 'Anteil Teilnehmer (%)')]

# Entferne Spalten mit '*'
df = df[df.columns[~df.columns.str.contains(r'\*')]]
df.columns = df.columns.str.replace('*', '', regex=False)

# Korrigiere einen Tippfehler
df = df.rename(columns={'Master Getränketechnologi': 'Master Getränketechnologie'})


# Extrahiere die Anzahl Teilnahmen an Studierendenbefragung
anzahl_studierende = df[df['Category'] == 'Anzahl Teilnahmen an Studierendenbefragung'].iloc[:, 2:].values.flatten()

# Extrahiere die Studienbeeinträchtigungen (alle Werte mit Kategorie 'Studienbeeinträchtigungen')
df_beeintraechtigungen = df[df['Variable'] == 'Studienbeeinträchtigungen'].iloc[:, 2:].astype(float)
# Berechne die relativen Werte
df_relativ = df_beeintraechtigungen.multiply(anzahl_studierende, axis=1) / 100

# Category-Spalte wieder hinzufügen
df_relativ.insert(0, 'Category', df[df['Variable'] == 'Studienbeeinträchtigungen']['Category'].values)

# Konvertiere die relevante Spalte in float (und setze ungültige Werte auf NaN)
df['Bachelor Agrarwissenschaften'] = pd.to_numeric(df['Bachelor Agrarwissenschaften'], errors='coerce')


df_relativ = df_relativ.fillna(value=0)
df_relativ['sum'] = df_relativ.sum(axis=1, numeric_only=True)
df_relativ = df_relativ.loc[~df_relativ['Category'].isin(['Anzahl Teilnahmen an Studierendenbefragung', 'Anteil Teilnehmer (%)']), ['Category', 'sum']]


/var/folders/gf/ldtpmxs55h916v2hnlzdv8zw0000gn/T/ipykernel_90860/2554228.py:34: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [57]:
df_relativ

,Category,sum
50,zeitliche Überschneidungen (%),102.18
51,Stundenplanprobleme wegen weiter Wege (%),21.09
52,Organisation des Prüfungsablaufes (%),88.21
53,Engpässe in der Literaturversorgung (%),24.99
54,kein Labor- / Praktikumsplatz bekommen (%),12.51
55,Anforderungen sind zu hoch (%),30.74
56,Leistungsnachweise / Prüfungen nicht geschafft...,41.51
57,Job / Erwerbstätigkeit (%),173.08
58,familiäre Verpflichtungen (%),72.09
59,Krankheit / Behinderung (%),67.50


In [58]:
# Entferne das Zeichen ' (%)' aus allen Einträgen in der 'Category'-Spalte
df_relativ['Category'] = df_relativ['Category'].str.replace(' (%)', '', regex=False)


In [59]:
umbenennungen = {
    'zeitliche Überschneidungen': 'Zeitmanagement-Probleme',
    'Stundenplanprobleme wegen weiter Wege': 'Wegeprobleme',
    'Organisation des Prüfungsablaufes': 'Prüfungsorganisation',
    'Engpässe in der Literaturversorgung': 'Literaturmangel',
    'kein Labor- / Praktikumsplatz bekommen': 'Fehlender Praktikums-/Laborplatz',
    'Anforderungen sind zu hoch': 'Überforderung',
    'Leistungsnachweise / Prüfungen nicht geschafft': 'Prüfungsprobleme',
    'Job / Erwerbstätigkeit': 'Erwerbstätigkeit',
    'familiäre Verpflichtungen': 'Familienpflichten',
    'Krankheit / Behinderung': 'Gesundheitsprobleme',
    'große Entfernung zwischen Wohnort und JLU': 'Entfernung zum Studienort',
    'Doppelstudium': 'Doppelte Studienbelastung',
    'Nachholen von erforderlichen Vorkenntnissen': 'Vorkenntnisnachholung',
    'fehlende soziale Kontakte': 'Isolation',
    'Benachteiligung': 'Ungleichbehandlung',
    'fehlende Berufsperspektiven': 'Karriereunsicherheit'
}

# Erstelle ein DataFrame
df_relativ_ = pd.DataFrame(df_relativ, columns=['Category'])

# Ersetze die Werte in der Spalte 'Category' basierend auf dem Dictionary
df_relativ['Category'] = df_relativ['Category'].replace(umbenennungen)


In [60]:
import plotly.express as px

# Deine bestehende Plotly Plot
fig = px.line_polar(df_relativ, r=df_relativ['sum'], theta=df_relativ['Category'], line_close=True)

# Traces aktualisieren
fig.update_traces(fill='toself')


fig.update_layout(
    template='infoviz',
    title=f'Beeinträchtigende Faktoren im Studiums des FB09 in {year}',
    polar=dict(
        radialaxis=dict(showticklabels=False),
       
    )
)



# Plot anzeigen
fig.show()
